# Simulation of an Electric System based on Thermoelectrics Plants

In [73]:
from src.map import Map2D, GraphMap
from src.utils.gaussianmixture import DailyElectricityConsumptionBimodal
from src.circuits import Circuit, Block
from src.thermoelectrics import Thermoelectric
from src.part import Coils, Boiler, Generator, SteamTurbine
import pandas as pd
import plotly.express as px
from numpy import random

from src.simulation_constants import (
    NO_CIRCUITS,
    NO_THERMOELECTRICS,
    MIN_CITIZEN,
    MAX_CITIZEN,
    MAX_DEVIATION_CITIZEN_IN_BLOCK,
    DEMAND_PER_PERSON,
    DEMAND_INDUSTRIALIZATION,
    VARIABILITY_DEMAND_PER_PERSON,
    VARIABILITY_DEMAND_PER_INDUSTRIALIZATION,
    PEAK_CONSUMPTION_MORNING,
    PEAK_CONSUMPTION_EVENING,
    MAX_DEVIATION_MORNING,
    MAX_DEVIATION_EVENING,
    WEIGHT_MORNING,
    WEIGHT_EVENING,
    MIN_BLOCKS_PER_CIRCUIT,
    MAX_BLOCKS_PER_CIRCUIT,
    IMPORTANCE_ALPHA,
    RANDOM_SEED,
    DISTANCE_REGULATOR
)

SIMULATION_INTERNAL_RANDOM = random.default_rng(RANDOM_SEED)

## Initialize bases 

In [74]:
map_2d = Map2D(
    no_circuits=NO_CIRCUITS,
    no_thermoelectrics=NO_THERMOELECTRICS,
)


map_2d.visualize()

In [75]:
graphMap = GraphMap(
    thermoelectric_labels=[f"Th{i}" for i in range(NO_THERMOELECTRICS)],
    circuits_labels=[f"C{i}" for i in range(NO_CIRCUITS)],
    towers_labels=[f"Tw{i}" for i in range(len(map_2d.towers_positions))],
    thermoelectrics_positions=map_2d.thermoelectrics_positions,
    circuits_positions=map_2d.circuits_positions,
    towers_positions=map_2d.towers_positions,
)

graphMap.visualize()

In [76]:
distance_cost_template = graphMap.thermoelectric_generation_cost

# Generate circuits for map

electric_system_circuits_list: list[Circuit] = []
for i in range(NO_CIRCUITS):
    citizen_count = SIMULATION_INTERNAL_RANDOM.integers(MIN_CITIZEN, MAX_CITIZEN)
    citizen_range = (
        max(citizen_count - MAX_DEVIATION_CITIZEN_IN_BLOCK, 0),
        min(citizen_count + MAX_DEVIATION_CITIZEN_IN_BLOCK, MAX_CITIZEN),
    )

    industrialization = SIMULATION_INTERNAL_RANDOM.integers(0, DEMAND_INDUSTRIALIZATION) / DEMAND_INDUSTRIALIZATION
    
    bimodal_consumption = DailyElectricityConsumptionBimodal(
        base_consumption=DEMAND_PER_PERSON * citizen_count
        + DEMAND_INDUSTRIALIZATION * industrialization,
        base_variability=VARIABILITY_DEMAND_PER_PERSON * citizen_count
        + VARIABILITY_DEMAND_PER_INDUSTRIALIZATION * industrialization,
        mean_morning=PEAK_CONSUMPTION_MORNING,
        mean_evening=PEAK_CONSUMPTION_EVENING,
        std_morning=SIMULATION_INTERNAL_RANDOM.uniform(1.0, MAX_DEVIATION_MORNING),
        std_evening=SIMULATION_INTERNAL_RANDOM.uniform(1.0, MAX_DEVIATION_EVENING),
        weight_morning=WEIGHT_MORNING,
        weight_evening=WEIGHT_EVENING,
    )

    electric_system_circuits_list.append(
        Circuit(
            graphMap.circuits_nodes[i].id,
            gaussian_mixture=bimodal_consumption,
            blocks_range=(MIN_BLOCKS_PER_CIRCUIT, MAX_BLOCKS_PER_CIRCUIT),
            citizens_range=citizen_range,
            industrialization=industrialization,
        )
    )

# Generate Thermoelectric generation based in the nearest circuits
mapper_circuit_with_thermoelectric = {}

for c in electric_system_circuits_list:

    filtered = [f for f in graphMap.thermoelectric_generation_cost if f[1] == c.id]
    filtered = sorted(filtered, key=lambda x: x[2])
    mapper_circuit_with_thermoelectric[c.id] = filtered[0][0]


import plotly.graph_objects as go

# Create edges for the plotly graph
edges = []
edges_labels = []
for circuit, thermoelectric in mapper_circuit_with_thermoelectric.items():

    circuitPos = [c.position for c in graphMap.circuits_nodes if c.id == circuit][0]
    thermoelectricPos = [
        t.position for t in graphMap.thermoelectrics_nodes if t.id == thermoelectric
    ][0]

    edges.append((circuitPos, thermoelectricPos))
    edges_labels.append(f"{circuit} -> {thermoelectric}")


# Create a plotly graph
fig = go.Figure()

# Add edges to the plotly graph
for i, edge in enumerate(edges):
    circuit, thermoelectric = edge
    fig.add_trace(
        go.Scatter(
            x=[circuit[0], thermoelectric[0]],
            y=[circuit[1], thermoelectric[1]],
            mode="lines",
            line=dict(color="black"),
            name="Wire Connection",
            showlegend=False,
            hoverinfo="text",
            text=edges_labels[i],
            hoverlabel=dict(bgcolor="white", font_size=16, font_family="Rockwell"),
        )
    )

# Add nodes to the plotly graph
all_nodes = graphMap.circuits_nodes + graphMap.thermoelectrics_nodes
for i, node in enumerate(all_nodes):
    fig.add_trace(
        go.Scatter(
            x=[node.position[0]],
            y=[node.position[1]],
            mode="markers+text",
            marker=dict(
                size=10, color="blue" if i >= len(graphMap.circuits_nodes) else "red"
            ),
            text=[node.id],
            textposition="top center",
            hoverinfo="text",
        )
    )

fig.update_layout(
    showlegend=False,
    title="Circuits and Thermoelectrics",
    height=800,
    plot_bgcolor="lightgreen",
)

fig.show()

In [77]:
# Generate the distance matrix


def distance_template_to_distance_matrix(
    template: list[tuple[str, str, float, list[str]]],
    thermoelectrics: list[str],
    circuits: list[str],
):
    matrix = [[-1 for _ in range(len(circuits))] for _ in range(len(thermoelectrics))]

    c_map = {}
    t_map = {}

    for i, t in enumerate(thermoelectrics):
        t_map[t] = i

    for i, c in enumerate(circuits):
        c_map[c] = i

    max_cost = 1
    for t, c, cost, _ in template:
        matrix[t_map[t]][c_map[c]] = cost
        max_cost = max(max_cost, cost)

    for t, c, _, _ in template:
        matrix[t_map[t]][c_map[c]] /= max_cost * DISTANCE_REGULATOR

    return matrix


matrix = distance_template_to_distance_matrix(
    distance_cost_template,
    [t.id for t in graphMap.thermoelectrics_nodes],
    [c.id for c in graphMap.circuits_nodes],
)


distance_matrix_df = pd.DataFrame(
    matrix,
    index=[t.id for t in graphMap.thermoelectrics_nodes],
    columns=[c.id for c in graphMap.circuits_nodes],
)

distance_matrix_df

,C0,C1,C2,C3,C4,C5,C6,C7,C8,C9,...,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49
Th0,0.004552,0.008274,0.008962,0.003300,0.003707,0.000352,0.001133,0.001885,0.000370,0.000494,...,0.008982,0.000929,0.000104,0.002405,0.006930,0.005746,0.000247,0.007013,0.000221,0.003826
Th1,0.002903,0.000986,0.001673,0.004331,0.003903,0.007742,0.008524,0.005691,0.007330,0.007885,...,0.001693,0.008320,0.007495,0.005363,0.000755,0.001785,0.007638,0.000664,0.007612,0.003653
Th2,0.003189,0.006912,0.007600,0.001938,0.002344,0.001763,0.002544,0.000523,0.001351,0.001905,...,0.007619,0.002340,0.001515,0.001042,0.005567,0.004384,0.001658,0.005650,0.001632,0.002463
Th3,0.000188,0.003911,0.004598,0.001436,0.001009,0.004848,0.005629,0.002796,0.004436,0.004990,...,0.004618,0.005426,0.004600,0.002469,0.002566,0.001382,0.004744,0.002649,0.004717,0.000759
Th4,0.005570,0.009293,0.009980,0.004319,0.004725,0.001090,0.000271,0.002904,0.001389,0.001024,...,0.010000,0.000110,0.001122,0.003423,0.007948,0.006765,0.001266,0.008031,0.001066,0.004844
Th5,0.002894,0.006617,0.007304,0.001642,0.002049,0.002402,0.003183,0.000350,0.001990,0.002544,...,0.007324,0.002979,0.002154,0.000747,0.005272,0.004088,0.002297,0.005355,0.002271,0.002168
Th6,0.003621,0.000175,0.000863,0.005048,0.004621,0.008460,0.009241,0.006408,0.008048,0.008603,...,0.000882,0.009038,0.008212,0.006081,0.001473,0.002502,0.008356,0.001381,0.008329,0.004371
Th7,0.000955,0.004677,0.005365,0.000537,0.000110,0.003949,0.004730,0.001897,0.003537,0.004091,...,0.005384,0.004527,0.003701,0.001570,0.003332,0.002149,0.003845,0.003415,0.003818,0.000229


In [78]:
# Generate thermoelectrics
electric_system_thermoelectrics_list: list[Thermoelectric] = []

for i, t in enumerate(graphMap.thermoelectrics_nodes):
    generated_thermoelectric_min_cost = 0

    for j, c in enumerate(graphMap.circuits_nodes):
        if mapper_circuit_with_thermoelectric[c.id] == t.id:
            generated_thermoelectric_min_cost += (
                electric_system_circuits_list[j].mock_electric_consume
                + electric_system_circuits_list[j].mock_electric_consume
                * matrix[i][j]
                * 24
            )

    electric_system_thermoelectrics_list.append(
        Thermoelectric(
            id=t.id,
            total_capacity=generated_thermoelectric_min_cost + 200,
        )
    )
    thermoelectric_data = {
        "ID": [t.id for t in electric_system_thermoelectrics_list],
        "Total Capacity": [
            t.total_capacity for t in electric_system_thermoelectrics_list
        ],
    }

    thermoelectric_df = pd.DataFrame(thermoelectric_data)


sorted_electric_system_circuit_list = sorted(
    electric_system_circuits_list,
    key=lambda x: list(
        filter(
            lambda y: mapper_circuit_with_thermoelectric[x.id] == y.id,
            graphMap.thermoelectrics_nodes,
        )
    )[0].id,
)

colors = [
    list(
        filter(
            lambda x, y=circuit: mapper_circuit_with_thermoelectric[y.id] == x.id,
            graphMap.thermoelectrics_nodes,
        )
    )[0].id
    for circuit in sorted_electric_system_circuit_list
]


fig = px.bar(
    thermoelectric_df,
    x="ID",
    y="Total Capacity",
    title="Total Capacity of Thermoelectrics",
    height=800,
    color=[t.id for t in electric_system_thermoelectrics_list],
    color_discrete_sequence=px.colors.qualitative.Plotly,
    labels={"color": "Thermoelectric ID"},
)

fig.show()

In [79]:

mock_electric_consume_data = {
    "Circuit ID": [c.id for c in sorted_electric_system_circuit_list],
    "Mock Electric Consume": [c.mock_electric_consume for c in sorted_electric_system_circuit_list],
}

mock_electric_consume_df = pd.DataFrame(mock_electric_consume_data)

fig = px.bar(
    mock_electric_consume_df,
    x="Circuit ID",
    y="Mock Electric Consume",
    title="Electric Consume per Circuit [24 hours]",
    height=800,
    color=colors
)
fig.show()

In [80]:
max_population_of_circuits = -1
max_population_of_block = -1

for circuit in electric_system_circuits_list:
    max_population_of_circuits = max(
        circuit.get_all_block_population(), max_population_of_circuits
    )
    for block in circuit.blocks:
        max_population_of_block = max(block.citizens.amount, max_population_of_block)

auxiliary_data_max_population_of_circuits = max_population_of_circuits
auxiliary_data_max_population_of_block = max_population_of_block


def get_circuit_importance(circuit: Circuit) -> float:
    return (
        circuit.get_all_block_population() / auxiliary_data_max_population_of_circuits
    ) * IMPORTANCE_ALPHA + circuit.industrialization * (1 - IMPORTANCE_ALPHA)


def get_block_importance(block: Block) -> float:
    return (
        block.citizens.amount / auxiliary_data_max_population_of_circuits
    ) * IMPORTANCE_ALPHA + block.industrialization * (1 - IMPORTANCE_ALPHA)


def set_importance(ci: list[Circuit]):
    for circuit in ci:
        for block in circuit.blocks:
            block.importance = get_block_importance(block)
        circuit.importance = get_circuit_importance(circuit)

set_importance(electric_system_circuits_list)

max_population_data = {
    "Max Population of Circuits": [auxiliary_data_max_population_of_circuits],
    "Max Population of Blocks": [auxiliary_data_max_population_of_block],
}

max_population_df = pd.DataFrame(max_population_data)
# TODO : update plotly
max_population_df

,Max Population of Circuits,Max Population of Blocks
0,19284,4979


In [81]:
# All data of one block of a circuit (importance, citizens, industrialization, demand per hour)

circuit_stats = electric_system_circuits_list[0]

blocks = circuit_stats.blocks

citizens = []
importance = []
demand_per_hour = []

for block in blocks:
    citizens.append(block.citizens.amount)
    importance.append(block.importance)
    demand_per_hour.append(sum(block.predicted_demand_per_hour))

block_stats_df = pd.DataFrame(
    {
        "Citizens": citizens,
        "Importance": importance,
        "Demand per Hour": demand_per_hour,
    }
)

fig_citizens = px.bar(
    block_stats_df,
    x=block_stats_df.index,
    y="Citizens",
    title=f"Block Citizens in Circuit {circuit_stats.id}",
    height=800,
)

fig_importance = px.bar(
    block_stats_df,
    x=block_stats_df.index,
    y="Importance",
    title=f"Block Importance in Circuit {circuit_stats.id}",
    height=800,
)

fig_demand_per_hour = px.bar(
    block_stats_df,
    x=block_stats_df.index,
    y="Demand per Hour",
    title=f"Block Demand per Hour in Circuit {circuit_stats.id}",
    height=800,
)


fig_citizens.show()
fig_importance.show()
fig_demand_per_hour.show()

In [82]:
# Compare data from circuits

citizens = []
importance = []
demand_per_hour = []

for circuit in sorted_electric_system_circuit_list:
    
    importance.append(circuit.importance)
    demand_per_hour.append(
        sum([sum(block.predicted_demand_per_hour) for block in circuit.blocks])
    )

# Separate for properties
citizens = [
    sum(block.citizens.amount for block in circuit.blocks)
    for circuit in electric_system_circuits_list
]

circuit_comparison_df = pd.DataFrame(
    {   "Circuit ID": [c.id for c in sorted_electric_system_circuit_list], 
        "Citizens": citizens,
        "Importance": importance,
        "Demand per Hour": demand_per_hour,
    }
)

fig_citizens = px.bar(
    circuit_comparison_df,
    x="Circuit ID",
    y="Citizens",
    title="Total Citizens per Circuit",
    height=800,
    color=colors,
    color_discrete_sequence=px.colors.qualitative.Plotly,
    labels={"color": "Thermoelectric ID"},
)

fig_importance = px.bar(
    circuit_comparison_df,
    x="Circuit ID",
    y="Importance",
    title="Importance per Circuit",
    height=800,
    color=colors,
    color_discrete_sequence=px.colors.qualitative.Plotly,
    labels={"color": "Thermoelectric ID"},
)

fig_demand_per_hour = px.bar(
    circuit_comparison_df,
    x="Circuit ID",
    y="Demand per Hour",
    title="Total Demand per Circuit in a Day",
    height=800,
    color=colors,
    color_discrete_sequence=px.colors.qualitative.Plotly,
    labels={"color": "Thermoelectric ID"},
)

fig_citizens.show()
fig_importance.show()
fig_demand_per_hour.show()

In [83]:
circuit_id = "C0"
circuit = next(c for c in electric_system_circuits_list if c.id == circuit_id)

block_demand_data = {"Block ID": [], "Hour": [], "Demand": []}

for i, block in enumerate(circuit.blocks):
    for hour, demand in enumerate(block.predicted_demand_per_hour):
        block_demand_data["Block ID"].append(f"Block {i + 1}")
        block_demand_data["Hour"].append(hour)
        block_demand_data["Demand"].append(demand)

block_demand_df = pd.DataFrame(block_demand_data)

fig_block_demand = px.line(
    block_demand_df,
    x="Hour",
    y="Demand",
    color="Block ID",
    title=f"Hourly Demand per Block in Circuit {circuit_id}",
    labels={"Demand": "Demand (kWh)", "Hour": "Hour"},
)

fig_block_demand.show()

circuit_demand_data = {
    "Hour": list(range(24)),
    "Demand": [
        sum(block.predicted_demand_per_hour[hour] for block in circuit.blocks)
        for hour in range(24)
    ],
}

circuit_demand_df = pd.DataFrame(circuit_demand_data)

fig_circuit_demand = px.line(
    circuit_demand_df,
    x="Hour",
    y="Demand",
    title=f"Hourly Demand of Circuit {circuit_id}",
    labels={"Demand": "Demand (kWh)", "Hour": "Hour"},
)

fig_circuit_demand.show()

# Simulation

In [84]:
from src.people import ThermoelectricAgent, ThermoelectricAgentPerception
from src.worldstate import WorldState


class Simulation:
    def __init__(
        self,
        circuits: list[Circuit],
        thermoelectrics: list[Thermoelectric],
        distance_matrix: list[list[float]],
        get_circuit_importance: callable,
        get_block_importance: callable,
    ):
        self.world_state_manager = WorldState(
            circuits=circuits,
            thermoelectrics=thermoelectrics,
            distance_matrix=distance_matrix,
            get_block_importance=get_block_importance,
            get_circuit_importance=get_circuit_importance,
        )

        self.thermoelectric_agents: list[ThermoelectricAgent] = []

    def create_thermoelectric_agent_perception(
        self, thermoelectric
    ) -> ThermoelectricAgentPerception:

        return ThermoelectricAgentPerception(
            thermoelectric=thermoelectric,
            general_deficit=self.world_state_manager.general_deficit,
            general_demand=self.world_state_manager.general_demand,
            general_offer=self.world_state_manager.general_offer,
        )

    def simulate(self, simulation_days=50):

        for thermoelectric in self.world_state_manager.thermoelectrics:
            agent_initial_perception = self.create_thermoelectric_agent_perception(thermoelectric=thermoelectric)
            print("agent perception")
            print(agent_initial_perception)
            thermoelectric_agent = ThermoelectricAgent(
                name=f"Agent of {thermoelectric.id}",
                thermoelectric=thermoelectric,
                perception=agent_initial_perception,
            )

            self.thermoelectric_agents.append(thermoelectric_agent)

    # check progress

    # for agent in self.thermoelectric_agents:
    #     print(agent)


simulation = Simulation(
    circuits=electric_system_circuits_list,
    thermoelectrics=electric_system_thermoelectrics_list,
    distance_matrix=matrix,
    get_circuit_importance=get_circuit_importance,
    get_block_importance=get_block_importance,
)

simulation.simulate()

agent perception
[]
[]
{
            "thermoelectric: {'id': 'Th0', 'total_capacity': 41378.080892371814, 'current_capacity': 0, 'stored_energy': 0, 'parts': []},
            "plant_status": False,
            "parts_status": [],
            "broken_parts": [],
            "max_capacity": 41378.080892371814,
            "current_capacity": 0,
            "power_output_reduction_on_part_failure": [],
            "general_deficit": 0,
            "general_demand": 0,
            "general_offer": 0,
        }
agent perception
[]
[]
{
            "thermoelectric: {'id': 'Th1', 'total_capacity': 18389.600700749175, 'current_capacity': 0, 'stored_energy': 0, 'parts': []},
            "plant_status": False,
            "parts_status": [],
            "broken_parts": [],
            "max_capacity": 18389.600700749175,
            "current_capacity": 0,
            "power_output_reduction_on_part_failure": [],
            "general_deficit": 0,
            "general_demand": 0,
            "gener